In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Concatenate
from CNN_PY import cnn_model
from LSTM_PY import lstm_model
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import keras
import joblib
import ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.preprocessing import image
from PIL import Image
import mplfinance as mpf
import os
from datetime import timedelta
import time
from tqdm import tqdm
import csv

# def create_meta_fusion_model():
#     # 입력: [LSTM 변화율, CNN 상승 확률, CNN 하락 확률]
#     inputs = layers.Input(shape=(3,))  # 입력 피처 3개
#     x = layers.Dense(32, activation='relu')(inputs)  # 은닉층 32 노드
#     x = layers.Dense(16, activation='relu')(x)       # 은닉층 16 노드
#     output = layers.Dense(1, activation='tanh')(x)   # 출력층: -1 ~ 1 (하락 ~ 상승)
    
#     model = models.Model(inputs, output)
#     return model

# meta_model = create_meta_fusion_model()
# meta_model.summary()



# cnn_model('2025-05-12', 'AAPL')
# lstm_model_1 = tf.keras.layers.TFSMLayer('LSTM_MODEL_TF_GPU/AOS', call_endpoint='serving_default')
# lstm_mode_2 = tf.keras.models.load_model(os.path.join('LSTM_MODEL_H5_WIN', 'AOS.h5'), compile=False)

In [3]:
symbol = 'AAPL'
date = '2024-05-14'
close = lstm_model(date, symbol)
cnn_output = cnn_model(date, symbol)
label = cnn_output[0]
prob = float(cnn_output[1])
actual_close =  111 #yfinance에서 받아와야함
print(symbol)
print(date)
print(close)
print(label)
print(prob)
print(actual_close)
file_path = 'MLP_sp500.csv'

with open(file_path, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([symbol, date, close, label, prob, actual_close])


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
AAPL
2024-05-14
180.89638493603647
neutral
0.9252076745033264
111


In [11]:
import pandas as pd

# 전체 S&P500 최신 데이터 로딩
df_all = pd.read_csv('sp500_latest.csv', parse_dates=['Date'])

# 원하는 기간 필터 (2024-01-01 ~ 2024-12-31)
df_filtered = df_all[(df_all['Date'] >= '2024-01-01') & (df_all['Date'] < '2025-01-01')]

# 필요한 칼럼 생성 및 초기화
df_mlp_init = df_filtered[['Symbol', 'Date']].copy()
df_mlp_init['Close'] = ''  # 예측값 저장용 → 초기 빈값
df_mlp_init['Label'] = ''  # 예측 Label → 빈값
df_mlp_init['Prob'] = ''   # 예측 확률 → 빈값
df_mlp_init['Actual_Close'] = df_filtered['Close']  # 실제 종가만 채워 넣음

# 저장
df_mlp_init.to_csv('MLP_sp500.csv', index=False)
print("✅ MLP_sp500.csv 초기화 완료 (Close 비워지고, Actual_Close만 채워짐)")


✅ MLP_sp500.csv 초기화 완료 (Close 비워지고, Actual_Close만 채워짐)


In [1]:
# multi_modal.ipynb
import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import datetime, timedelta
from tqdm import tqdm
from CNN_PY import cnn_model
from LSTM_PY import lstm_model

# 1. 공통 설정
SYMBOLS_FILE = 'completed_symbols_h5.txt'
OUTPUT_CSV = 'MLP_sp500_temp.csv'
START_DATE = '2024-01-01'
END_DATE = '2025-01-01'

# 2. 종목 리스트 로드
with open(SYMBOLS_FILE, 'r') as f:
    symbols = [line.strip() for line in f if line.strip()]

# 3. 기존 데이터 로드
if os.path.exists(OUTPUT_CSV):
    mlp_df = pd.read_csv(OUTPUT_CSV)
    existing_keys = set(zip(mlp_df['Symbol'], mlp_df['Date']))
else:
    mlp_df = pd.DataFrame(columns=['Symbol','Date','Close','Label','Prob','Actual_Close'])
    existing_keys = set()

# 4. 종목별 처리
for symbol in tqdm(symbols, desc="종목 처리 중"):
    try:
        # 1회만 데이터 다운로드 (추가 40일 버퍼 포함)
        full_df = yf.download(
            symbol, 
            start=pd.to_datetime(START_DATE)-timedelta(days=40),
            end=pd.to_datetime(END_DATE)+timedelta(days=2),
            progress=False
        )
        
        if full_df.empty:
            continue
            
        # 컬럼 정리
        full_df = full_df[['Open','High','Low','Close','Volume']].dropna()
        full_df['Date_Str'] = full_df.index.strftime('%Y-%m-%d')
        
        # 대상 기간 필터링 (2024-01-01 ~ 2025-01-01)
        target_df = full_df.loc[START_DATE:END_DATE].copy()
        
        for date_str, row in target_df.iterrows():
            date_str = date_str.strftime('%Y-%m-%d')
            
            # 중복 체크
            if (symbol, date_str) in existing_keys:
                continue
                
            # LSTM 예측 (전체 데이터 전달)
            close_pred = lstm_model(date_str, symbol, full_df)
            if close_pred is None:
                continue
                
            # CNN 예측 (전체 데이터 전달)
            label, prob = cnn_model(date_str, symbol, full_df)
            if label is None:
                continue
                
            # 결과 저장
            new_row = {
                'Symbol': symbol,
                'Date': date_str,
                'Close': close_pred,
                'Label': label,
                'Prob': prob,
                'Actual_Close': row['Close']
            }
            mlp_df = pd.concat([mlp_df, pd.DataFrame([new_row])], ignore_index=True)
            existing_keys.add((symbol, date_str))
            
    except Exception as e:
        print(f"\n{symbol} 처리 실패: {str(e)}")
        continue

# 5. 결과 저장
mlp_df.to_csv(OUTPUT_CSV, index=False)
print(f"\n처리 완료! 총 {len(mlp_df)}개 데이터 저장됨")

종목 처리 중:   0%|          | 0/497 [00:00<?, ?it/s]

YF.download() has changed argument auto_adjust default to True
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step


종목 처리 중:   0%|          | 0/497 [00:04<?, ?it/s]

CNN 예측 오류 (MMM): Data for column "Open" must be ALL float or int.


KeyboardInterrupt: 

In [5]:
# MLP 학습할 데이터셋 구축 (ORIGINAL)

import pandas as pd
import numpy as np
import yfinance as yf
import csv
import os
from datetime import datetime, timedelta
from CNN_PY import cnn_model
from LSTM_PY import lstm_model
import time

# 파일 경로
symbols_file = 'completed_symbols_h5.txt'
output_csv = 'MLP_sp500_temp.csv'

# 기간
start_date = '2024-01-01'
end_date = '2025-01-01'

# 초기 CSV 파일 존재 확인 및 중복 키 확보
if os.path.exists(output_csv):
    existing_df = pd.read_csv(output_csv)
    existing_keys = set(existing_df.apply(lambda x: f"{x['Symbol']}_{x['Date']}", axis=1))
else:
    # 파일 없으면 헤더 생성
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Symbol', 'Date', 'Close', 'Label', 'Prob', 'Actual_Close'])
    existing_keys = set()

# 종목 리스트 로드
with open(symbols_file, 'r') as f:
    symbols = [line.strip() for line in f.readlines() if line.strip()]

# 종목별 처리
for symbol in symbols:
    print(f"\n### {symbol} 처리 시작 ###")
    try:
        # 한번만 데이터 다운로드 (전체 거래일 확보)
        df = yf.download(symbol, start=start_date, end=end_date, interval='1d', progress=False)
        if df.empty:
            print(f"{symbol}: 기간 내 데이터 없음")
            continue
        
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [col[0] for col in df.columns]

        df = df.dropna(subset=['Close']).copy()
        df['Date_Str'] = df.index.strftime('%Y-%m-%d')

        # 거래일별 처리
        for idx, row in df.iterrows():
            date = row['Date_Str']
            key = f"{symbol}_{date}"

            if key in existing_keys:
                print(f"{symbol} {date}: 이미 존재, 스킵")
                continue

            try:
                # LSTM 예측 Close
                close = lstm_model(date, symbol)
                if close is None:
                    print(f"{symbol} {date}: LSTM 예측 실패")
                    continue

                # CNN 예측 Label, Prob
                label, prob = cnn_model(date, symbol)
                if label is None or prob is None:
                    print(f"{symbol} {date}: CNN 예측 실패")
                    continue

                # 실제 종가 (다운로드된 df에서 사용)
                actual_close = row['Close']

                print(f"{symbol} | {date} | Close(LSTM): {close:.2f} | Label: {label} | Prob: {prob:.4f} | Actual_Close: {actual_close:.2f}")

                # CSV 기록
                with open(output_csv, 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([symbol, date, close, label, prob, actual_close])

                existing_keys.add(key)

            except Exception as e:
                print(f"{symbol} {date} 처리 오류: {e}")

            time.sleep(0.5)  # 딜레이 (yfinance는 이미 다운로드됨)

    except Exception as e:
        print(f"{symbol} 전체 처리 오류: {e}")

print("\n### 전체 완료 ###")




### MMM 처리 시작 ###
MMM 2024-01-02: 이미 존재, 스킵
MMM 2024-01-03: 이미 존재, 스킵
MMM 2024-01-04: 이미 존재, 스킵
MMM 2024-01-05: 이미 존재, 스킵
MMM 2024-01-08: 이미 존재, 스킵
MMM 2024-01-09: 이미 존재, 스킵
MMM 2024-01-10: 이미 존재, 스킵
MMM 2024-01-11: 이미 존재, 스킵
MMM 2024-01-12: 이미 존재, 스킵
MMM 2024-01-16: 이미 존재, 스킵
MMM 2024-01-17: 이미 존재, 스킵
MMM 2024-01-18: 이미 존재, 스킵
MMM 2024-01-19: 이미 존재, 스킵
MMM 2024-01-22: 이미 존재, 스킵
MMM 2024-01-23: 이미 존재, 스킵
MMM 2024-01-24: 이미 존재, 스킵


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step


[*********************100%***********************]  1 of 1 completed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


MMM | 2024-02-07 | Close(LSTM): 76.56 | Label: surge | Prob: 0.7602 | Actual_Close: 75.46


[*********************100%***********************]  1 of 1 completed


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step


KeyboardInterrupt: 

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers
import joblib

# 1. Load dataset (MLP_sp500.csv)
df = pd.read_csv('MLP_sp500.csv')

# 2. Ensure actual close (y) is present
features = ['Close', 'Label', 'Prob']
X = df[features].values
y = df['Actual_Close'].values  # 반드시 'Actual_Close' 열이 있어야 함

# 3. Scale inputs (입력만 Scaling, y는 그대로)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 5. Build MLP model (Regression)
mlp = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # 종가 예측
])

mlp.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 6. Train
mlp.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

# 7. Save model & scaler
mlp.save('MLP_model.h5')
joblib.dump(scaler, 'MLP_scaler.joblib')

# 8. Evaluate
loss, mae = mlp.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {loss:.4f}, MAE: {mae:.4f}")


Epoch 1/50


c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5625/5625 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 255666.9844 - mae: 205.4474 - val_loss: 256762.9531 - val_mae: 204.3289
Epoch 2/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 655us/step - loss: 249867.2812 - mae: 201.1551 - val_loss: 254814.1562 - val_mae: 199.5109
Epoch 3/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 3s 589us/step - loss: 250188.3594 - mae: 194.4481 - val_loss: 252923.3594 - val_mae: 194.8001
Epoch 4/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 3s 618us/step - loss: 267611.2500 - mae: 192.6266 - val_loss: 251086.3438 - val_mae: 190.2311
Epoch 5/50
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 4s 659us/step - loss: 246117.8906 - mae: 187.2902 - val_loss: 249303.1875 - val_mae: 185.8461
Epoch 6/50
  93/5625 ━━━━━━━━━━━━━━━━━━━━ 3s 625us/step - loss: 265030.0625 - mae: 179.5308

KeyboardInterrupt: 

In [ ]:
# # Mehtod 1


# # 1. 모델과 스케일러 불러오기
# # LSTM
# lstm_model = keras.layers.TFSMLayer('LSTM_MODEL_TF_GPU/AOS', call_endpoint='serving_default')
# lstm_scaler = joblib.load('LSTM_MODEL_TF_GPU/AOS_scaler.joblib')
# # CNN
# cnn_model = tf.keras.models.load_model.load_model('pattern_classification_model.h5')

# # 2. 메타 입력 생성
# lstm_out = lstm_model.output  # shape: (None, 1)
# cnn_out = cnn_model.layers[-2].output  # 마지막 레이어 직전 추출 (3D 특징 맵)
# cnn_flatten = keras.layers.Flatten()(cnn_out)  # shape: (None, 2048)

# # 3. 결합 및 MLP 처리
# combined = Concatenate()([lstm_out, cnn_flatten])
# x = keras.layers.Dense(64, activation='relu')(combined)
# x = keras.layers.Dense(32, activation='relu')(x)
# final_output = keras.layers.Dense(1, activation='linear')(x)

# # 4. 최종 모델 컴파일
# meta_model = keras.layers.Model(inputs=[lstm_model.input, cnn_model.input], outputs=final_output)
# meta_model.compile(optimizer='adam', loss='mse')

In [ ]:
# # Method 2


# # LSTM 모델 및 스케일러 불러오기 (학습 가능한 형태로 재구성 필요할 수 있음)
# lstm_model_loaded = keras.models.load_model('LSTM_MODEL_TF_GPU/AOS') # SavedModel 형식에 따라 로드 방식이 다를 수 있음
# lstm_scaler = joblib.load('LSTM_MODEL_TF_GPU/AOS_scaler.joblib')

# # CNN 모델 불러오기
# cnn_model_loaded = tf.keras.models.load_model('pattern_classification_model.h5')

# # Meta-Model 학습 데이터 생성 함수 (가정)
# def generate_multimodal_data(symbols, start_date, end_date, n_lstm_days=50, n_cnn_candles=20):
#     lstm_predictions = []
#     cnn_predictions = []
#     actual_close_changes = [] # 예시: 다음날 종가 변화율
#     # ... (데이터 생성 로직 구현: 각 종목, 날짜별 LSTM 예측, CNN 예측, 실제 종가 변화 수집)
#     # 이 함수는 실제 데이터와 모델 예측을 기반으로 학습 데이터를 생성해야 합니다.
#     # 예시로 임의의 데이터를 생성합니다.
#     num_samples = 1000
#     lstm_predictions = np.random.rand(num_samples, 1) * 200 # 임의의 종가 예측
#     cnn_probabilities = np.random.rand(num_samples, 3) # 임의의 [drop, neutral, surge] 확률
#     actual_close_changes = np.random.randn(num_samples, 1) * 0.05 # 임의의 종가 변화율
#     return np.concatenate([lstm_predictions, cnn_probabilities], axis=1), actual_close_changes

# # Meta-Model 구축
# def build_meta_model(lstm_output_dim, cnn_output_dim):
#     lstm_input = Input(shape=(lstm_output_dim,))
#     cnn_input = Input(shape=(cnn_output_dim,))
#     merged = concatenate([lstm_input, cnn_input])
#     dense1 = Dense(64, activation='relu')(merged)
#     dense2 = Dense(32, activation='relu')(dense1)
#     output = Dense(1)(dense2) # 최종 종가 변화율 예측
#     meta_model = Model(inputs=[lstm_input, cnn_input], outputs=output)
#     return meta_model

# # 학습 데이터 생성 (실제 데이터 기반으로 구현 필요)
# symbols = ['AAPL', 'MSFT']
# start_date = '2020-01-01'
# end_date = '2024-12-31'
# X_meta, y_meta = generate_multimodal_data(symbols, start_date, end_date)

# # Meta-Model 입력 형태 정의
# lstm_output_dim = 1 # LSTM 예측 종가
# cnn_output_dim = 3 # CNN 예측 확률 (drop, neutral, surge)

# # Meta-Model 구축
# meta_model = build_meta_model(lstm_output_dim, cnn_output_dim)

# # Meta-Model 컴파일
# meta_model.compile(optimizer='adam', loss='mse')

# # 학습 데이터 분할
# X_train, X_test, y_train, y_test = train_test_split(X_meta, y_meta, test_size=0.2, random_state=42)
# lstm_input_train = X_train[:, :lstm_output_dim]
# cnn_input_train = X_train[:, lstm_output_dim:]
# lstm_input_test = X_test[:, :lstm_output_dim]
# cnn_input_test = X_test[:, lstm_output_dim:]

# # Meta-Model 학습
# meta_model.fit([lstm_input_train, cnn_input_train], y_train, epochs=10, batch_size=32, validation_data=([lstm_input_test, cnn_input_test], y_test))

# # Meta-Model 평가
# loss = meta_model.evaluate([lstm_input_test, cnn_input_test], y_test)
# print(f"Meta-Model Test Loss: {loss}")

# # Meta-Model 예측
# predictions = meta_model.predict([lstm_input_test, cnn_input_test])
# print("Meta-Model Predictions:")
# print(predictions[:5])
# print("Actual Changes:")
# print(y_test[:5])